In [1]:
import os
import sys

from dotenv import load_dotenv

sys.path.append(os.path.abspath(".."))

load_dotenv("/Users/steffen/agentic_ai/.env")

True

In [2]:
from src.agent.adapters.sql_tools.controller import create_sql_controller

controller = create_sql_controller()

In [3]:
questions = [
    "How many customers do we have?",
    "What are the top 5 selling products?",
    "Show me orders from 2024",
    "What is the average order value?",
]


In [20]:
i = 3

In [21]:
result = controller.process_question(questions[i])

2025-07-03 23:11:00.686 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-03 23:11:00.687 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-03 23:11:00.760 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [23]:
result.confidence

1.0

In [24]:
print(result.sql_query)

SELECT AVG(grand_total_amount) AS average_order_value
FROM orders;
